# 영화리뷰 텍스트 감성분석기

이번에는 글 속에 담긴 감성을 분석하는 모델을 구현해보겠습니다. 사람도 글에 담긴 뉘앙스를 파악하는게 쉽지 않은데 기계가 할 수 있다는게 너무 신기하네요.

다른 사람의 마음을 알고 싶다는 인간의 욕구는 어제오늘 일이 아니었죠. 특히 인터넷 상에 올라온 글들은 특정한 대상을 가지고 있는 경우가 많고, 그 대상을 향한 감정을 분석함으로써 효율적이고 많은 수의 대상에 대한 평가를 확보할 수 있게 되었습니다. 이러한 정보는 마케팅 등의 영역에서는 매우 귀중한 정보가 될 수 있겠죠.

이러한 정보를 얻기 위한 단계인 **감성분석**(*Sentiment Analysis*)은 어떠한 **대상**(*Target*)을 향한 **감정표현**(*Sentiment Expression*)이 필수적이며 이로부터 감정(emotion), 감성(sentiment), 태도(attitude), 입장(stance) 등을 분석한다.

분석은 총 3단계로 이루어지는데, **데이터 수집 -> 주관성 탐지 -> 주관의 정도**로 나누기 순서로 분류하는데, 주관의 정도를 나눌 때 **기계학습 기반 접근법**과 **감성사전 기반 접근법** 두 가지 형태로 접근할 수 있다.  기계학습은 라벨링이 되어있는 문서나 문장으로 학습한 모델을 사용하는 것이며 이 방식이 불가능할 경우 감성사전 기반 분석 방법을 사용하게 되는데, 감성사전 기반 분석은 미리 구성된 감성사전의 품질이 결과로 이어진다. 문제는 양질의 감성사전을 구축하는 일은 많은 시간과 비용이 들어가는 점, 분석 대상에 따라 점수가 달라질 수 있다는 점, 애매한 긍부정은 분석이 어렵다는 점 등의 단점이 있습니다.

그럼 대체 왜 텍스트 데이터일까요? 텍스트 데이터는 SNS등 인터넷 상에서 대량의 데이터를 쉽게 확보가 가능한데, 이 데이터들에는 **작성자의 주관**이 그대로 드러나 있고 **실시간 트랜드**를 빠르게 반영하는 특징이 있기 때문인데요. 문제는 텍스트는 구조가 희소하고 차원의 수가 크기 때문에 전통적인 지도학습으로는 좋은 성과를 내기 힘들었습니다. 그런데 최근에는 텍스트를 저차원의 벡터값으로 표현하는 **워드 임베딩**(*word embedding*)기법이 등장해서 **CNN**(*Convolutional Neural Network*)과 **LSTM**(*Long Short-Term Memory Network*) 등의 딥러닝 모델에 활용되고 있습니다.

## 1. 자료준비하기

### (1) 텍스트 데이터를 숫자로 표현하기

컴퓨터는 단순히 0과 1만을 입력받습니다. 그러면 텍스트 데이터를 입력하려면 이를 숫자로 변환해줘야겠죠.


